In [10]:
# ╭──────────────────────────────────────────────╮
# │ 1. 加载 Qwen API Key                         │
# ╰──────────────────────────────────────────────╯

%load_ext dotenv
%dotenv

import os
from dotenv import load_dotenv
load_dotenv()

QWEN_API_KEY = os.getenv("QWEN_API_KEY")
if not QWEN_API_KEY:
    raise ValueError("❌ 请在 .env 中配置 QWEN_API_KEY")

print("✅ Qwen API Key 已加载！")

✅ Qwen API Key 已加载！


In [2]:
# ╭──────────────────────────────────────────────╮
# │ 2. 共享工具（所有 Agent 可用）               │
# ╰──────────────────────────────────────────────╯

import datetime
import re

def get_current_time() -> str:
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression: str) -> str:
    if not re.match(r'^[\d\s\+\-\*\/ $$\.]+ $ ', expression):
        return "ERROR"
    try:
        return str(eval(expression, {"__builtins__": {}}, {}))
    except:
        return "ERROR"

TOOLS = {
    "get_current_time": get_current_time,
    "calculate": calculate
}

TOOL_DESC = "可用工具：get_current_time(), calculate(expr)"

In [4]:
# ╭──────────────────────────────────────────────╮
# │ 3. 基础协作 Agent                            │
# │ 每个 Agent 有：角色、目标、记忆、工具        │
# ╰──────────────────────────────────────────────╯

from openai import OpenAI
import json

class CollaborativeAgent:
    def __init__(self, name: str, role: str, goal: str, api_key: str, model: str = "qwen-max"):
        self.name = name
        self.role = role
        self.goal = goal
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
        self.memory = []  # 存储接收到的消息
    
    def receive_message(self, sender: str, content: str):
        """接收来自其他 Agent 或用户的消息"""
        self.memory.append({"from": sender, "content": content})
    
    def generate_reply(self, instruction: str = "") -> str:
        """
        基于记忆和指令生成回复
        """
        context = "\n".join([
            f"[{msg['from']}]: {msg['content']}" 
            for msg in self.memory[-5:]  # 只看最近5条
        ])
        
        prompt = f"""你是一个名为 {self.name} 的智能体，角色是「{self.role}」，目标是：{self.goal}。当前对话上下文：{context}
                    {instruction} 请以 {self.name} 的身份直接回复，不要解释."""
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        ).choices[0].message.content
        
        return response
    
    def clear_memory(self):
        self.memory = []

In [6]:
# ╭──────────────────────────────────────────────╮
# │ 3. 基础协作 Agent                            │
# │ 每个 Agent 有：角色、目标、记忆、工具        │
# ╰──────────────────────────────────────────────╯

from openai import OpenAI
import json

class CollaborativeAgent:
    def __init__(self, name: str, role: str, goal: str, api_key: str, model: str = "qwen-max"):
        self.name = name
        self.role = role
        self.goal = goal
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
        self.memory = []  # 存储接收到的消息
    
    def receive_message(self, sender: str, content: str):
        """接收来自其他 Agent 或用户的消息"""
        self.memory.append({"from": sender, "content": content})
    
    def generate_reply(self, instruction: str = "") -> str:
        """
        基于记忆和指令生成回复
        """
        context = "\n".join([
            f"[{msg['from']}]: {msg['content']}" 
            for msg in self.memory[-5:]  # 只看最近5条
        ])
        
        prompt = f"""你是一个名为 {self.name} 的智能体，角色是「{self.role}」，目标是：{self.goal}。当前对话上下文：
                    {context}{instruction} 请以 {self.name} 的身份直接回复，不要解释。""".strip()
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        ).choices[0].message.content
        
        return response
    
    def clear_memory(self):
        self.memory = []

In [11]:
# ╭──────────────────────────────────────────────╮
# │ 4. 多 Agent 协作控制器                       │
# │                                              │
# │ 流程：                                       │
# │   1. 用户提问 → 分配给初始 Agent             │
# │   2. Agent 回复 → 决定是否转交他人           │
# │   3. 最多 N 轮后汇总输出                     │
# ╰──────────────────────────────────────────────╯

class MultiAgentCollaboration:
    def __init__(self, agents: list, max_turns: int = 6):
        self.agents = {agent.name: agent for agent in agents}
        self.max_turns = max_turns
    
    def chat(self, user_question: str) -> str:
        print(f"👤 用户: {user_question}\n")
        
        # 清空所有 Agent 记忆
        for agent in self.agents.values():
            agent.clear_memory()
        
        # 初始消息：发给第一个 Agent（如 Writer）
        current_agent = list(self.agents.values())[0]
        current_agent.receive_message("User", user_question)
        
        conversation_log = [("User", user_question)]
        
        for turn in range(self.max_turns):
            # 当前 Agent 生成回复
            reply = current_agent.generate_reply()
            print(f"🤖 {current_agent.name} ({current_agent.role}): {reply}")
            conversation_log.append((current_agent.name, reply))
            
            # 判断是否需要转交（简单规则：包含 "@Reviewer" 等）
            if "@Reviewer" in reply and "Reviewer" in self.agents:
                next_agent = self.agents["Reviewer"]
                next_agent.receive_message(current_agent.name, reply)
                current_agent = next_agent
            elif "@Executor" in reply and "Executor" in self.agents:
                next_agent = self.agents["Executor"]
                next_agent.receive_message(current_agent.name, reply)
                current_agent = next_agent
            elif "FINAL ANSWER:" in reply:
                # 提取最终答案
                final = reply.split("FINAL ANSWER:")[-1].strip()
                print(f"\n🎯 最终答案: {final}")
                return final
            else:
                # 默认继续由当前 Agent 处理（或结束）
                pass
            
            # 防止无限循环
            if turn == self.max_turns - 1:
                break
        
        # 超时，由最后一个 Agent 总结
        summary = current_agent.generate_reply("请总结以上讨论，给出最终答案。")
        print(f"\n⚠️ 超过最大轮数，强制总结:")
        print(f"🎯 最终答案: {summary}")
        return summary

In [12]:
# ╭──────────────────────────────────────────────╮
# │ 5. 定义具体角色 + 测试协作                   │
# ╰──────────────────────────────────────────────╯

# 创建三个 Agent
writer = CollaborativeAgent(
    name="Writer",
    role="内容撰写者",
    goal="根据问题起草清晰、准确的回答草稿",
    api_key=QWEN_API_KEY
)

reviewer = CollaborativeAgent(
    name="Reviewer",
    role="质量审核员",
    goal="检查草稿的准确性、逻辑性和安全性，必要时要求修改或调用工具",
    api_key=QWEN_API_KEY
)

executor = CollaborativeAgent(
    name="Executor",
    role="工具执行者",
    goal="当需要外部信息时，调用工具（如计算、时间）并返回结果",
    api_key=QWEN_API_KEY
)

# 启动协作系统
collab = MultiAgentCollaboration([writer, reviewer, executor], max_turns=5)

# 测试复杂问题
print("="*60)
result = collab.chat("今天是几号？2025年还有多少天？请分步计算并审核。")

👤 用户: 今天是几号？2025年还有多少天？请分步计算并审核。

🤖 Writer (内容撰写者): 今天是2023年11月29日。首先，从今天到2023年年底还有32天（包括今天）。接着，2024年为闰年，共有366天。因此，从现在到2025年底总共有32 + 366 + 365 = 763天。这意味着距离2025年结束还有763天。

审核：上述计算基于当前日期为2023年11月29日的前提。考虑到不同地区可能存在时差影响实际的“今天”定义，请根据实际情况调整起始日期以获得最准确的结果。
🤖 Writer (内容撰写者): 今天是2023年11月28日。

计算2025年剩余天数步骤如下：
1. 识别2025年的总天数。由于2025年能够被4整除但不能被100整除，除非同时能被400整除（这里不适用），因此2025年是一个闰年，共有366天。
2. 计算从今天到2025年底的天数。首先确定从今天到2024年底的总天数，然后加上2025年的全部天数。
   - 2023年剩余天数：30(12月) - 28(已过天数) = 2天
   - 2024年为闰年，共366天
   - 2025年全年共366天
3. 总计：2 + 366 + 366 = 734天

综上所述，从今天起至2025年年底还有734天。请注意，此计算基于当前日期为2023年11月28日进行。
🤖 Writer (内容撰写者): 今天是2023年10月5日。

计算2025年剩余天数步骤如下：

1. 从2023年10月5日至2023年末共有\(31-5+30+31+30+31+31+29=248\)天（假设非闰年，即2月份有28天）。
2. 2024年为闰年，因此全年共366天。
3. 2025年直到目标日期前的天数需要具体指定哪一天作为结束点来计算；但若要计算整个2025年的天数，则为365天。

综上所述，从今天到2025年底总共有\(248 + 366 + 365 = 979\)天。

审核：考虑到2024年是闰年，上述计算中包含了额外的一天。对于“2025年还有多少天”的部分，如果没有特别指明具体的结束日期，默认理解为询问整个2025年度的天数，则答案应直接为365天。如果意图了解从当前日期至2025年底的具体天数，则上述计算方法适用。
🤖 Writer (内容撰写者): 今天是202

In [ ]:
# ╭──────────────────────────────────────────────╮
# │ 6. 项目全景回顾                              │
# ╰──────────────────────────────────────────────╯

print("🎉 恭喜！你已完成整个 LLM Agent 学习路径：")

steps = [
    "01. 调通 Qwen API",
    "02. 调通 Doubao API",
    "03. 统一接口（理解抽象）",
    "04. 多模型编排（控制流）",
    "05. 添加记忆（上下文）",
    "06. 接入工具（行动力）",
    "07. ReAct 推理（自主规划）",
    "08. 多 Agent 协作（团队智能）"
]

for i, step in enumerate(steps, 1):
    print(f"{i}. {step}")

print("\n💡 你现在掌握的能力：")
print("• 从零构建可观察、可控制的 Agent")
print("• 理解 LangChain 等框架背后的原理")
print("• 具备扩展到真实应用的基础")

print("\n🚀 下一步建议：")
print("• 将 Agent 部署为 Web 服务（FastAPI + Streamlit）")
print("• 接入向量数据库实现长期记忆")
print("• 尝试用你的 Agent 自动写周报、查数据、做分析！")

print("\n❤️ 记住：")
print("真正的智能，不在于模型有多强，")
print("而在于你如何设计它的‘思考’与‘行动’方式。")